### Импорты

In [1]:
import os
import sys
sys.path.insert(0, '../')
import pandas as pd

from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
from colbert import Indexer, Searcher

### Подготовка моделей

In [2]:
pth_models = "/home/sondors/Documents/price/ColBERT_data/18_categories/test/models_18_categories.csv"
df_models = pd.read_csv(pth_models, sep=";")
df_models

/tmp/ipykernel_23022/17807022.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_models = pd.read_csv(pth_models, sep=";")


,model_id,average_price,name,brand_name,full_name,comment,category_name,category_id
0,623742,NaN,920-005619,Logitech,Logitech 920-005619,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
1,721952,NaN,Zipper Bag,Hama,Hama Zipper Bag,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
2,721970,NaN,CC-3064,Nokia,Nokia CC-3064,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
3,751488,990.0,CKS-X7/R,Sony,Sony CKS-X7/R,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
4,751989,NaN,EP-031023,Era Pro,Era Pro EP-031023,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
...,...,...,...,...,...,...,...,...
103209,7049424,16459.0,MD-108,Mivo,Mivo MD-108,NaN,портативная акустика,3904
103210,7049425,8812.0,MD-165,Mivo,Mivo MD-165,NaN,портативная акустика,3904
103211,7049426,4240.0,Boost 20W,Rocket,Rocket Boost 20W,NaN,портативная акустика,3904
103212,7049427,2990.0,Motion 10W,Rocket,Rocket Motion 10W,NaN,портативная акустика,3904


In [3]:
def prepare_tsv(df, pth, category_id):
    """
    Делим модели по full_name и соответствующие им model_id на два tsv файла 

    {category_id}_models.tsv:

    0   model0
    1   model1
    2   model2

    {category_id}_models_id.tsv:

    0   model_id0
    1   model_id1
    2   model_id2
    
    """

    def df_split(df):
        df1 = pd.DataFrame()
        df1["id"], df1["full_name"] = [i for i in range(len(df))], df["full_name"]
        
        df2 = pd.DataFrame()
        df2["id"], df2["model_id"] = [i for i in range(len(df))], df["model_id"]

        return df1, df2
    
    models, models_id = df_split(df)
    models.to_csv(os.path.join(pth,f"{category_id}_models.tsv"), sep='\t', header=False, index=False)
    models_id.to_csv(os.path.join(pth,f"{category_id}_models_id.tsv"), sep='\t', header=False, index=False)

categories = [
    "диктофоны, портативные рекордеры",
    "электронные книги",
    "автомобильные телевизоры, мониторы",
    "смарт-часы и браслеты",
    "портативные медиаплееры",
    # "чехлы, обложки для гаджетов (телефонов, планшетов etc)",
    "портативная акустика",
    "мобильные телефоны",
    "VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)",
    "планшетные компьютеры и мини-планшеты",
    "наушники, гарнитуры, наушники c микрофоном",
    "радиоприемники, радиобудильники, радиочасы",
    "магнитолы",
    "GPS-навигаторы"
    ]

categories_id = [
    3902,
    510402,
    4302,
    2815,
    3901,
    # 3994,
    3904,
    2801,
    3908,
    510401,
    2102,
    3903,
    3907,
    280801
    ]

dst_fld = "/home/sondors/Documents/price/ColBERT/tmp_tutorial"
for cat_id in categories_id:
    category_models = df_models[df_models.category_id == cat_id].reset_index(drop=True)
    prepare_tsv(category_models, dst_fld, cat_id)

### Индексируем модели и сохраняем индекс на диск

In [5]:
def save_index(ckpt_pth, doc_maxlen, nbits, nranks, experiment, collection, index_name):
    with Run().context(RunConfig(nranks=nranks, experiment=experiment)):
        config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits)
        indexer = Indexer(checkpoint=ckpt_pth, config=config)
        indexer.index(name=index_name, collection=collection, overwrite=True)
    return indexer

ckpt_pth = "/home/sondors/Documents/ColBERT_weights/bert-base-multilingual-cased_dim_768_bsize_230_lr04_use_ib_negatives/none/2024-01/27/16.55.29/checkpoints/colbert-2998-finish"
experiment = "tutorial"

doc_maxlen = 300
nbits = 2   # encode each dimension with 2 bits
nranks = 1  # nranks specifies the number of GPUs to use.

for cat_id in categories_id:

    models_colbert = Collection(path=os.path.join(dst_fld,f"{cat_id}_models.tsv"))
    index_name = f'models_{cat_id}_{nbits}bits'
    indexer = save_index(ckpt_pth, doc_maxlen, nbits, nranks, experiment, models_colbert, index_name)

[Feb 02, 22:21:39] #> Loading collection...
0M 


[Feb 02, 22:21:39] #> Creating directory /home/sondors/Documents/price/ColBERT/experiments/tutorial/indexes/models_3902_2bits 


#> Starting...
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "nbits": 2,
    "kmeans_niters": 4,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 0.0001,
    "maxsteps": 2998,
    "save_every": null,
    "warmup": 0,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 768,
    "doc_maxlen": 300,
    "mask_punctuat

/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
  0%|          | 0/8 [00:00<?, ?it/s]/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Feb 02, 22:21:43] [0] 		 # of sampled PIDs = 488 	 sampled_pids[:3] = [213, 375, 5]
[Feb 02, 22:21:43] [0] 		 #> Encoding 488 passages..


100%|██████████| 8/8 [00:04<00:00,  1.96it/s]
WARNING clustering 4691 points to 1024 centroids: please provide at least 39936 training points
0it [00:00, ?it/s]
  0%|          | 0/8 [00:00<?, ?it/s]

[Feb 02, 22:21:47] [0] 		 avg_doclen_est = 10.116803169250488 	 len(local_sample) = 488
[Feb 02, 22:21:47] [0] 		 Creaing 1,024 partitions.
[Feb 02, 22:21:47] [0] 		 *Estimated* 4,936 embeddings.
[Feb 02, 22:21:47] [0] 		 #> Saving the indexing plan to /home/sondors/Documents/price/ColBERT/experiments/tutorial/indexes/models_3902_2bits/plan.json ..
Clustering 4691 points in 768D to 1024 clusters, redo 1 times, 4 iterations
  Preprocessing in 0.00 s
  Iteration 3 (0.08 s, search 0.07 s): objective=392.74 imbalance=1.505 nsplit=0        
[0.009, 0.008, 0.01, 0.007, 0.011, 0.008, 0.01, 0.01, 0.009, 0.009, 0.008, 0.009, 0.008, 0.007, 0.008, 0.008, 0.01, 0.008, 0.01, 0.008, 0.009, 0.01, 0.009, 0.009, 0.008, 0.011, 0.008, 0.008, 0.007, 0.009, 0.01, 0.01, 0.008, 0.009, 0.009, 0.009, 0.008, 0.008, 0.009, 0.007, 0.011, 0.009, 0.009, 0.01, 0.01, 0.009, 0.008, 0.008, 0.007, 0.007, 0.009, 0.007, 0.009, 0.008, 0.008, 0.008, 0.008, 0.01, 0.009, 0.007, 0.009, 0.007, 0.009, 0.01, 0.008, 0.008, 0.008, 

...

[Feb 02, 22:33:40] [0] 		 avg_doclen_est = 9.476302146911621 	 len(local_sample) = 1,709
[Feb 02, 22:33:40] [0] 		 Creaing 1,024 partitions.
[Feb 02, 22:33:40] [0] 		 *Estimated* 16,195 embeddings.
[Feb 02, 22:33:40] [0] 		 #> Saving the indexing plan to /home/sondors/Documents/price/ColBERT/experiments/tutorial/indexes/models_280801_2bits/plan.json ..
Clustering 15386 points in 768D to 1024 clusters, redo 1 times, 4 iterations
  Preprocessing in 0.01 s
  Iteration 3 (0.22 s, search 0.21 s): objective=3269.42 imbalance=1.504 nsplit=0       
[0.015, 0.014, 0.015, 0.013, 0.016, 0.013, 0.013, 0.014, 0.013, 0.016, 0.012, 0.012, 0.012, 0.013, 0.012, 0.013, 0.013, 0.013, 0.014, 0.013, 0.013, 0.015, 0.012, 0.014, 0.014, 0.015, 0.011, 0.011, 0.012, 0.014, 0.014, 0.015, 0.013, 0.012, 0.013, 0.011, 0.013, 0.012, 0.013, 0.012, 0.014, 0.014, 0.014, 0.014, 0.014, 0.011, 0.012, 0.012, 0.012, 0.013, 0.013, 0.012, 0.014, 0.013, 0.013, 0.012, 0.013, 0.016, 0.012, 0.011, 0.014, 0.012, 0.013, 0.013, 0.01

0it [00:00, ?it/s]
100%|██████████| 27/27 [00:15<00:00,  1.78it/s]


[Feb 02, 22:33:55] [0] 		 #> Saving chunk 0: 	 1,709 passages and 16,195 embeddings. From #0 onward.
[Feb 02, 22:33:56] [0] 		 #> Checking all files were saved...
[Feb 02, 22:33:56] [0] 		 Found all files!
[Feb 02, 22:33:56] [0] 		 #> Building IVF...
[Feb 02, 22:33:56] [0] 		 #> Loading codes...
[Feb 02, 22:33:56] [0] 		 Sorting codes...
[Feb 02, 22:33:56] [0] 		 Getting unique codes...
[Feb 02, 22:33:56] #> Optimizing IVF to store map from centroids to list of pids..
[Feb 02, 22:33:56] #> Building the emb2pid mapping..
[Feb 02, 22:33:56] len(emb2pid) = 16195
[Feb 02, 22:33:56] #> Saved optimized IVF to /home/sondors/Documents/price/ColBERT/experiments/tutorial/indexes/models_280801_2bits/ivf.pid.pt
[Feb 02, 22:33:56] [0] 		 #> Saving the indexing metadata to /home/sondors/Documents/price/ColBERT/experiments/tutorial/indexes/models_280801_2bits/metadata.json ..


1it [00:15, 15.44s/it]
100%|██████████| 1024/1024 [00:00<00:00, 107151.84it/s]


#> Joined...


### Поиск матча по индексу

In [12]:
experiment = "tutorial"

doc_maxlen = 300
nbits = 2   # encode each dimension with 2 bits
nranks = 1  # nranks specifies the number of GPUs to use.
cat_id = 2801 # мобильные телефоны
cat_id = 510401 # планшеты
index_name = f'models_{cat_id}_{nbits}bits'
models_colbert = Collection(path=os.path.join(dst_fld,f"{cat_id}_models.tsv"))

with Run().context(RunConfig(experiment=experiment)):
    searcher = Searcher(index=index_name, collection=models_colbert)

[Feb 02, 23:06:16] #> Loading collection...
0M 
[Feb 02, 23:06:17] #> Loading codec...
[Feb 02, 23:06:17] #> Loading IVF...
[Feb 02, 23:06:17] #> Loading doclens...


/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
100%|██████████| 1/1 [00:00<00:00, 1781.78it/s]

[Feb 02, 23:06:17] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 288.65it/s]


In [13]:
offers = {
    0: 'Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)',
    1: 'Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)',
    2: 'Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)'
    }

offers = Queries(data=offers)
rankings = searcher.search_all(offers, k=5).todict()
rankings

/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')



#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global), 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([   101,    100,  37077,    524,  19079, 105694,  37077,  29723,  14248,
         10457,    156,  11396,    117,    129,    512,  18683,    120,  16196,
           512,  18683,    117,  52742,    118,  36448,    116,  29494,  18062,
           117,  10956,  67459,  19954,    102])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])



3it [00:00, 328.14it/s]


{0: [(3730, 1, 28.973285675048828),
  (3729, 2, 24.965259552001953),
  (2722, 3, 23.999181747436523),
  (3731, 4, 23.97533416748047),
  (2331, 5, 23.638072967529297)],
 1: [(3730, 1, 28.152374267578125),
  (3729, 2, 23.93535041809082),
  (2722, 3, 23.20758819580078),
  (1790, 4, 22.81966781616211),
  (2331, 5, 22.648759841918945)],
 2: [(3729, 1, 26.02865982055664),
  (3730, 2, 20.31949234008789),
  (3581, 3, 18.63184928894043),
  (2722, 4, 18.34659194946289),
  (3731, 5, 18.30866813659668)]}

In [22]:
for offer_index in rankings.keys():
    print(f"\noffer: {offers[offer_index]}")
    for passage_id, passage_rank, passage_score in rankings[offer_index]:
        print(f"\t [{passage_rank}] \t {passage_score:.1f} \t {searcher.collection[passage_id]}")


offer: Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)
	 [1] 	 29.0 	 Samsung Galaxy Tab S8
	 [2] 	 25.0 	 Samsung Galaxy Tab S8+
	 [3] 	 24.0 	 Samsung Galaxy Tab S2 9.7 SM-T813
	 [4] 	 24.0 	 Samsung Galaxy Tab S8 Ultra
	 [5] 	 23.6 	 Samsung Galaxy Tab S2 9.7 SM-T810

offer: Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)
	 [1] 	 28.2 	 Samsung Galaxy Tab S8
	 [2] 	 23.9 	 Samsung Galaxy Tab S8+
	 [3] 	 23.2 	 Samsung Galaxy Tab S2 9.7 SM-T813
	 [4] 	 22.8 	 Samsung Galaxy Tab S 8.4 SM-T705
	 [5] 	 22.6 	 Samsung Galaxy Tab S2 9.7 SM-T810

offer: Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)
	 [1] 	 26.0 	 Samsung Galaxy Tab S8+
	 [2] 	 20.3 	 Samsung Galaxy Tab S8
	 [3] 	 18.6 	 Samsung Galaxy Tab S7+ 12.4 128Gb
	 [4] 	 18.3 	 Samsung Galaxy Tab S2 9.7 SM-T813
	 [5] 	 18.3 	 Samsung Galaxy Tab S8 Ultra


### Получаем выдачу топ N моделей для каждого оффера [переписать под colbert]

In [10]:
def top_n_similar(offer_embs: List[np.ndarray], model_embs: List[np.ndarray], model_ids: List[int], batch_size: int = 1000, n: int = 5) -> List[Dict[str, Union[List[int], np.ndarray]]]:
    """
    Find the top N similar embeddings for each offer embedding.

    Args:
        offer_embs (List[np.ndarray]): List of offer embeddings.
        model_embs (List[np.ndarray]): List of model embeddings.
        model_ids (List[int]): List of model IDs corresponding to model embeddings.
        batch_size (int, optional): Batch size for chunking the offer embeddings. Defaults to 1000.
        n (int, optional): Number of top similar embeddings to retrieve. Defaults to 5.

    Returns:
        List[Dict[str, Union[List[int], np.ndarray]]]: A list of dictionaries containing model IDs and their 
        corresponding cosine similarity scores for the top N similar embeddings for each offer embedding.

    Example:
        >>> offer_embeddings = [np.array([0.1, 0.2, 0.3]), np.array([0.4, 0.5, 0.6])]
        >>> model_embeddings = [np.array([0.2, 0.3, 0.4]), np.array([0.5, 0.6, 0.7])]
        >>> model_ids = [123, 456]
        >>> print(top_n_similar(offer_embeddings, model_embeddings, model_ids))
            # Output: [{'model_ids': [123, 456], 'cosine_sims': array([0.99258333, 0.96832966])},
            {'model_ids': [456, 123], 'cosine_sims': array([0.99964575, 0.99461155])}]
    """

    cosine_sims = cosine_similarity_batch(offer_embs, model_embs, batch_size)
    top_n_list = []
    for i in range(len(offer_embs)):
        # Find the indices of the top N similar embeddings
        top_n_indices = np.argsort(cosine_sims[i])[::-1][:n]
        # Get the corresponding model IDs and cosine similarity scores for the top N similar embeddings
        top_n_model_ids = [model_ids[i] for i in top_n_indices]
        top_n_cosine_sims = cosine_sims[i][top_n_indices]
        top_n_list.append({"model_ids": top_n_model_ids, "cosine_sims": top_n_cosine_sims})
    return top_n_list

top_n = top_n_similar(offer_embs, model_embs, model_ids, batch_size = 1000, n=3)

for i in range(len(top_n)):
    print(offers[i])
    for j in range(len(top_n[i]['model_ids'])):
        id = top_n[i]['model_ids'][j]
        sim = top_n[i]['cosine_sims'][j]
        print(f"\t{id}: {models[model_ids.index(id)]} --> {round(float(sim), 2)}")
    print("_"*60)

Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)
	5144478: Samsung Galaxy Tab S8 --> 0.89
	410416: Starway Andromeda S8 --> 0.78
	4509801: Samsung Galaxy Tab S7 11 128Gb --> 0.76
____________________________________________________________
Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)
	5144478: Samsung Galaxy Tab S8 --> 0.89
	4509801: Samsung Galaxy Tab S7 11 128Gb --> 0.77
	410416: Starway Andromeda S8 --> 0.77
____________________________________________________________
Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)
	5144477: Samsung Galaxy Tab S8+ --> 0.78
	631587: Haier G781-S --> 0.19
	4509801: Samsung Galaxy Tab S7 11 128Gb --> 0.15
____________________________________________________________


In [11]:
print(top_n)

[{'model_ids': [5144478, 410416, 4509801], 'cosine_sims': array([0.8934316, 0.7825217, 0.7603698], dtype=float32)}, {'model_ids': [5144478, 4509801, 410416], 'cosine_sims': array([0.88650525, 0.77400523, 0.76853645], dtype=float32)}, {'model_ids': [5144477, 631587, 4509801], 'cosine_sims': array([0.78279126, 0.18910483, 0.15151899], dtype=float32)}]
